In [3]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import numpy as np
import pandas as pd


from pymongo import MongoClient



###########################
# Data Manipulation / Model
###########################
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)

# readAll is the method name from AnimalShelter class and returns a cursor object
# update the next line to match your method name from your CRUD Python module class
df = pd.DataFrame.from_records(shelter.read({}))

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dash DataTable Only')



image_filename = 'GraziosoSalvareLogo.png' # customer image
encoded_image = base64.b64encode(open(image_filename, 'rb').read())

app.layout = html.Div([
    html.Center(html.B(html.H1('SNHU CS-340 Dashboard'))),
    html.Center(html.B(html.H2('Stuart Williams'))),
    html.Hr(),
    # customer image location, can you put it in different location and resize?
    html.Img(id='customer-image',src='data:image/png;base64,{}'.format(encoded_image.decode()),alt='customer image'),
    # Button location at top
    html.Div(className='row',
             style={'display' : 'flex'},
                  children=[
                          html.Button(id='submit-button-one', n_clicks=false, children='Water'),
                          html.Button(id='submit-button-two', n_clicks=false, children='Mountain'),
                          html.Button(id='submit-button-three', n_clicks=false, children='Disaster'),
                          html.Button(id='submit-button-four', n_clicks=true, children='Reset')
                  ]),
    dash_table.DataTable(
        id='datatable-interactivity',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        editable=False,
        filter_action="native",
        sort_action="native",
        sort_mode="multi",
        column_selectable=False,
        row_selectable=False,
        row_deletable=False,
        selected_columns=[],
        selected_rows=[],
        page_action="native",
        page_current= 0,
        page_size= 10,
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6'
        ),
    html.Hr(),
    html.H4('Percentage of Breeds'),
    dcc.Graph(id="graph")
])

#############################################
# Interaction Between Components / Controller
#############################################


@app.callback(Output('datatable-interactivity',"data"), 
              [Input('submit-button-one', 'n_clicks'),Input('submit-button-two', 'n_clicks'),
              Input('submit-button-three', 'n_clicks'),Input('submit-button-four', 'n_clicks')])
def on_click(bt1,bt2,bt3,bt4):
    # start case
    if (bt1):
        attrs = {"$or":[{"breed":"Labrador Retriever Mix"}, {"breed":"Chesapeake Bay Retriever"}
                        , {"breed":"Newfoundland"}], {"sex_upon_outcome":"Intact Female"}
                , "$and":[{"age_upon_outcome_in_weeks":{"$gt":26}},
                                 {"age_upon_outcome_in_weeks":{"$lt":156}}]}
    elif (bt2):
        attrs = {"$or":[{"breed":"German Shepherd"}, {"breed":"Alaskan Malamute"}
                        , {"breed":"Old English Sheepdog"}, {"breed":"Siberian Husky"}
                       , {"breed":"Rottweiler"}], {"sex_upon_outcome":"Intact Male"}
                , "$and":[{"age_upon_outcome_in_weeks":{"$gt":26}},
                                 {"age_upon_outcome_in_weeks":{"$lt":156}}]}
    elif (bt3):
        attrs = {"$or":[{"breed":"German Shepherd"}, {"breed":"Doberman Pinscher"}
                        , {"breed":"Golden Retriever"}, {"breed":"Bloodhound"}
                       , {"breed":"Rottweiler"}], {"sex_upon_outcome":"Intact Male"}
                , "$and":[{"age_upon_outcome_in_weeks":{"$gt":26}},
                                 {"age_upon_outcome_in_weeks":{"$lt":300}}]}
    elif (bt4):
        attrs = {}
    
    df = pd.DataFrame.from_records(shelter.read(attrs))
    return df.to_dict('records')


#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]

#map
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_viewport_data")])
def update_map(viewData):
    dff = pd.DataFrame.from_dict(viewData)
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            dl.Marker(position=[dff['location_long'].iloc[0],dff['location_lat'].iloc[0]], children=[
                dl.Tooltip(dff.iloc[0,4]),
                dl.Popup([
                    html.H1(df['breed'].iloc[0]),
                    html.P(dff.iloc[1,9])
                ])
            ])
        ])
    ]

@app.callback(
    Output("graph", "figure"), 
    Input('datatable-id', "derived_viewport_data"))
def generate_chart(viewData):
    df = pd.DataFrame.from_dict(viewData)
    fig = px.pie(df, values='breed', names='breed', hole=.3)
    return fig

app


In [1]:
from jupyter_plotly_dash import JupyterDash

import dash
import dash_leaflet as dl
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import dash_table
from dash.dependencies import Input, Output
import base64

import numpy as np
import pandas as pd


from pymongo import MongoClient



###########################
# Data Manipulation / Model
###########################
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter
# FIX ME update with your username and password and CRUD Python module name
username = "aacuser"
password = "password"
shelter = AnimalShelter(username, password)

# readAll is the method name from AnimalShelter class and returns a cursor object
# update the next line to match your method name from your CRUD Python module class
df = pd.DataFrame.from_records(shelter.read({}))
print(df)

                        _id     age_upon_outcome animal_id animal_type  \
0  623743698c11e2327c59bdaf   3          2 years   A716330         Dog   
1  623743698c11e2327c59bdb0   9          3 years   A720214         Dog   
2  623743698c11e2327c59bdb1  10         3 months   A664290         Cat   
3  623743698c11e2327c59bdb2  11           1 year   A721199         Dog   
4  623743698c11e2327c59bdb3   2           1 year   A725717         Cat   
5  623743698c11e2327c59bdb4  13           1 year   A700408         Cat   
6  623743698c11e2327c59bdb5  14          2 years   A742287         Dog   
7  623743698c11e2327c59bdb6  15          3 years   A712638         Dog   
8  623743698c11e2327c59bdb7  16          5 years   A723742         Dog   
9  623743698c11e2327c59bdb8  17         6 months   A668960         Dog   

                     breed                color date_of_birth  \
0  Chihuahua Shorthair Mix          Brown/White    2013-11-18   
1   Labrador Retriever Mix            Red/White    2013